In [43]:
#|default_exp 03.01_app

In [1]:
#|export
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)
import tsensor
# Get rid of findfont: Font family 'Arial' not found.
# when running a remote notebook on Jupyter Server on Ubuntu Linux server
import logging
logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)

from lolviz import *


mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations Version

### Basic Architecture

- Assign the number of items in our training set to `n` - where `n` is the number of training examples
- Assign the number of pixels in each item to `m` - where `m` is the number of pixels
- Assign possible values of digits to `c` - where `c` is the possible values of digits

In [4]:
x_train, x_train.shape

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([50000, 784]))

In [12]:
x_train[1, :].shape

torch.Size([784])

In [14]:
x_train[1,:].reshape(28,28).shape

torch.Size([28, 28])

In [15]:
#|export
n,m = x_train.shape
c = y_train.max()+1

- Assign to `nh` an integer for our 50 hidden layers, aka our 50 activation functinons, in this case *relus*

In [21]:
#|export
nh = 50

- Create the weight matrices for multiplication. 
    - The first set of weights, `w1` will be for our training set
    - The weights will also need a bias, `b1` that will begin as zeros 
    - The second set of weights, `w2` will be for our output matrix, which will just  be one output, which is the guess of what number it is
    - The second bias `b2` be initialized as zeros and match `w2`

In [22]:
n,m,c,nh

(50000, 784, tensor(10), 50)

In [50]:
#|export
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [51]:
w1.shape, b1.shape, w2.shape, b2.shape

(torch.Size([784, 50]), torch.Size([50]), torch.Size([50, 1]), torch.Size([1]))

In [90]:
(x_train@w1 + b1).shape

torch.Size([50000, 50])

- Create a function `lin` to perform matrix multiplication and add the bias 

In [78]:
#|export
def lin(x, w, b):
    return x@w + b

In [79]:
x_valid.shape

torch.Size([10000, 784])

In [80]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [81]:
t

tensor([[ -2.38,  18.71,  -8.72,  ...,  -0.74,  19.42,   6.90],
        [ -6.54,   7.11, -11.90,  ...,  -1.14,  -4.02,   6.22],
        [  3.42,  -0.41,   6.16,  ...,  -6.20,   0.67,  -4.74],
        ...,
        [  6.39,   2.75,   1.62,  ...,   0.32,   1.59,  -8.12],
        [ 13.71,  14.85,  -1.77,  ...,   0.24,   5.15,  -2.74],
        [ -1.58,   6.40,  -3.54,  ...,  11.86,  12.06,   6.59]])

- Create a `relu` function to flatten values at 0

In [82]:
#|export
def relu(x):
    return x.clamp_min(0.)

In [83]:
t = relu(t)
t

tensor([[ 0.00, 18.71,  0.00,  ...,  0.00, 19.42,  6.90],
        [ 0.00,  7.11,  0.00,  ...,  0.00,  0.00,  6.22],
        [ 3.42,  0.00,  6.16,  ...,  0.00,  0.67,  0.00],
        ...,
        [ 6.39,  2.75,  1.62,  ...,  0.32,  1.59,  0.00],
        [13.71, 14.85,  0.00,  ...,  0.24,  5.15,  0.00],
        [ 0.00,  6.40,  0.00,  ..., 11.86, 12.06,  6.59]])

- Create a `model` function to take a mini-batch `xs`
    - Create the first linear layer
    - Put first linear layer through a relu
    - Pass the relu-ed first linear layer through the second linear layer and return the value

In [84]:
#|export
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

- Pass the validation set through the model to check if working`

In [87]:
#|export
res = model(x_valid)
res

tensor([[ 37.33],
        [-11.00],
        [ 19.92],
        ...,
        [ 31.65],
        [-36.43],
        [ 20.06]])

### Loss Function MSE

- Subtract `y_valid` from `res` and think about how broadcasting rules apply here

In [44]:
import nbdev

In [67]:
nbdev.export.nb_export('03.01_backpropagation_rebuild.ipynb', '03.01_app')